In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

digits = load_digits()
X = digits.data
y_true = digits.target

rng = np.random.RandomState(0)

def mask_labeled(y_true, frac_labeled, rng):
    n = len(y_true)
    mask = rng.rand(n) < frac_labeled
    y_semi = np.full_like(y_true, -1)
    y_semi[mask] = y_true[mask]
    return y_semi, mask

full_clf = LogisticRegression(max_iter=1000)
full_clf.fit(X, y_true)

fractions = [0.01, 0.05, 0.10, 0.20]
n_clusters = 10


for frac in fractions:
    print(f"\n Procent etykietowanych: {frac:.2f}")

    # a) losowe ukrycie etykiet
    y_semi, labeled_mask = mask_labeled(y_true, frac, rng)
    unlabeled_mask = ~labeled_mask

    # b) K-Means – wspólne dla obu metod
    kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init=10)
    cluster_ids = kmeans.fit_predict(X)
    centroids = kmeans.cluster_centers_

    # metoda 1: głosowanie
    cluster_labels_vote = np.full(n_clusters, -1, dtype=int)

    for c in range(n_clusters):
        in_cluster = (cluster_ids == c) & labeled_mask
        if np.any(in_cluster):
            vals, counts = np.unique(y_true[in_cluster], return_counts=True)
            cluster_labels_vote[c] = vals[np.argmax(counts)]

    y_pred_vote = cluster_labels_vote[cluster_ids]

    valid_vote_mask = unlabeled_mask & (y_pred_vote != -1)
    if np.any(valid_vote_mask):
        acc_vote = accuracy_score(y_true[valid_vote_mask],
                                  y_pred_vote[valid_vote_mask])
    else:
        acc_vote = np.nan
    
    # metoda 2: centroidy
    labeled_idx = np.where(labeled_mask)[0]
    X_labeled = X[labeled_idx]
    y_labeled = y_true[labeled_idx]

    cluster_labels_centroid = np.empty(n_clusters, dtype=int)
    for c in range(n_clusters):
        dists = np.linalg.norm(X_labeled - centroids[c], axis=1)
        nearest_idx = np.argmin(dists)
        cluster_labels_centroid[c] = y_labeled[nearest_idx]

    # teraz każdy klaster ma etykietę – zero -1, zero głosowania
    y_pred_centroid = cluster_labels_centroid[cluster_ids]
    acc_centroid = accuracy_score(y_true[unlabeled_mask],
                                  y_pred_centroid[unlabeled_mask])

    # k-NN na etykietowanych
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X[labeled_mask], y_true[labeled_mask])
    y_knn = knn.predict(X[unlabeled_mask])
    acc_knn = accuracy_score(y_true[unlabeled_mask], y_knn)

    # pełny nadzorowany
    y_full_pred = full_clf.predict(X[unlabeled_mask])
    acc_full = accuracy_score(y_true[unlabeled_mask], y_full_pred)

    print(f"K-Means + głosowanie:                         {acc_vote:.4f}")
    print(f"K-Means + centroid:                           {acc_centroid:.4f}")
    print(f"k-NN:                                         {acc_knn:.4f}")
    print(f"Pełny nadzorowany (LogReg, 100% etykiet):     {acc_full:.4f}")



 Procent etykietowanych: 0.01
K-Means + głosowanie:                         0.6428
K-Means + centroid:                           0.4122
k-NN:                                           0.2333
Pełny nadzorowany (LogReg, 100% etykiet):     1.0000

 Procent etykietowanych: 0.05
K-Means + głosowanie:                         0.7717
K-Means + centroid:                           0.7248
k-NN:                                           0.8275
Pełny nadzorowany (LogReg, 100% etykiet):     1.0000

 Procent etykietowanych: 0.10
K-Means + głosowanie:                         0.7833
K-Means + centroid:                           0.7895
k-NN:                                           0.9003
Pełny nadzorowany (LogReg, 100% etykiet):     1.0000

 Procent etykietowanych: 0.20
K-Means + głosowanie:                         0.7857
K-Means + centroid:                           0.7857
k-NN:                                           0.9564
Pełny nadzorowany (LogReg, 100% etykiet):     1.0000


In [4]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

digits = load_digits()
X = digits.data
y_true = digits.target

rng = np.random.RandomState(0)

def mask_labeled(y_true, frac_labeled, rng):
    n = len(y_true)
    mask = rng.rand(n) < frac_labeled
    y_semi = np.full_like(y_true, -1)
    y_semi[mask] = y_true[mask]
    return y_semi, mask

full_clf = LogisticRegression(max_iter=1000)
full_clf.fit(X, y_true)

fractions = [0.01, 0.05, 0.10, 0.20]
n_clusters = 10


for frac in fractions:
    print(f"\n Procent etykietowanych: {frac:.2f}")

    # a) losowe ukrycie etykiet
    y_semi, labeled_mask = mask_labeled(y_true, frac, rng)
    unlabeled_mask = np.logical_not(labeled_mask)

    # b) K-Means
    kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init=10)
    cluster_ids = kmeans.fit_predict(X)
    centroids = kmeans.cluster_centers_

    # przypisanie etykiet klastrom przez głosowanie większościowe
    cluster_labels = np.full(n_clusters, -1, dtype=int)
    for c in range(n_clusters):
        in_cluster = (cluster_ids == c) & labeled_mask
        if np.any(in_cluster):
            vals, counts = np.unique(y_true[in_cluster], return_counts=True)
            cluster_labels[c] = vals[np.argmax(counts)]

    # c) 1. głosowanie
    y_pred_vote = cluster_labels[cluster_ids]
    valid_vote_mask = unlabeled_mask & (y_pred_vote != -1)

    if np.any(valid_vote_mask):
        acc_vote = accuracy_score(y_true[valid_vote_mask], y_pred_vote[valid_vote_mask])
    else:
        acc_vote = np.nan

    # c) 2. najbliższy centroid
    missing = cluster_labels == -1
    if np.any(missing):
        labeled_clusters = np.where(cluster_labels != -1)[0]
        for c in np.where(missing)[0]:
            dists = np.linalg.norm(centroids[labeled_clusters] - centroids[c], axis=1)
            nearest = labeled_clusters[np.argmin(dists)]
            cluster_labels[c] = cluster_labels[nearest]

    y_pred_nearest = cluster_labels[cluster_ids]
    acc_nearest = accuracy_score(y_true[unlabeled_mask], y_pred_nearest[unlabeled_mask])

    # kNN
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X[labeled_mask], y_true[labeled_mask])
    y_knn = knn.predict(X[unlabeled_mask])
    acc_knn = accuracy_score(y_true[unlabeled_mask], y_knn)

    # pełny nadzorowany
    y_full_pred = full_clf.predict(X[unlabeled_mask])
    acc_full = accuracy_score(y_true[unlabeled_mask], y_full_pred)

    print(f"K-Means + głosowanie:                         {acc_vote:.4f}")
    print(f"K-Means + głosowanie + najbliższy centroid:   {acc_nearest:.4f}")
    print(f"k-NN (tylko etykietowane):                    {acc_knn:.4f}")
    print(f"Pełny nadzorowany (LogReg, 100% etykiet):     {acc_full:.4f}")



 Procent etykietowanych: 0.01
K-Means + głosowanie:                         0.6428
K-Means + głosowanie + najbliższy centroid:   0.4033
k-NN (tylko etykietowane):                    0.2333
Pełny nadzorowany (LogReg, 100% etykiet):     1.0000

 Procent etykietowanych: 0.05
K-Means + głosowanie:                         0.7717
K-Means + głosowanie + najbliższy centroid:   0.7717
k-NN (tylko etykietowane):                    0.8275
Pełny nadzorowany (LogReg, 100% etykiet):     1.0000

 Procent etykietowanych: 0.10
K-Means + głosowanie:                         0.7833
K-Means + głosowanie + najbliższy centroid:   0.7833
k-NN (tylko etykietowane):                    0.9003
Pełny nadzorowany (LogReg, 100% etykiet):     1.0000

 Procent etykietowanych: 0.20
K-Means + głosowanie:                         0.7857
K-Means + głosowanie + najbliższy centroid:   0.7857
k-NN (tylko etykietowane):                    0.9564
Pełny nadzorowany (LogReg, 100% etykiet):     1.0000
